In [1]:

!python -m pip install pennylane numpy scipy pandas plotly tqdm nbformat



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 81.5 MB/s eta 0:00:00


In [3]:
import platform
import pennylane as qml
import numpy as np
import plotly

print("OS:", platform.system(), platform.release())
print("Python version:", platform.python_version())
print("PennyLane version:", qml.__version__)
print("NumPy version:", np.__version__)
print("Plotly version:", plotly.__version__)




OS: Linux 6.6.105+
Python version: 3.12.12
PennyLane version: 0.43.2
NumPy version: 2.0.2
Plotly version: 5.24.1


/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


## Measurement Theory Primer

### Born Rule
For a quantum state with density matrix ρ and a measurement operator Mₖ,
the probability of outcome k is given by:

p(k) = Tr(Mₖ ρ)

In projective measurements, the operators satisfy:
- Mₖ² = Mₖ
- ∑ₖ Mₖ = I

This notebook uses Pauli projective measurements along the X, Y, and Z
axes for single-qubit quantum state tomography.


### SIC POVM vs Pauli Projective Measurements (Single Qubit)

**SIC POVM**
- Informationally complete with only 4 outcomes
- Symmetric structure improves numerical stability
- Requires non-standard measurement bases
- More complex classical post-processing

**Pauli Projective Measurements**
- Measures in X, Y, Z bases
- Hardware-native and easy to interpret
- Requires multiple measurement settings
- Higher total shot cost

For this assignment, Pauli projective measurements were primarily used
for clarity and ease of implementation, while SIC POVMs are discussed
as a scalable alternative.


In [4]:
import numpy as np



Pz0 = np.array([[1, 0],
                [0, 0]])

Pz1 = np.array([[0, 0],
                [0, 1]])

Px0 = 0.5 * np.array([[1,  1],
                      [1,  1]])

Px1 = 0.5 * np.array([[1, -1],
                      [-1, 1]])

Py0 = 0.5 * np.array([[1, -1j],
                      [1j, 1]])

Py1 = 0.5 * np.array([[1,  1j],
                      [-1j, 1]])

pauli_measurements = {
    "Z": [Pz0, Pz1],
    "X": [Px0, Px1],
    "Y": [Py0, Py1]
}

print("Pauli measurement operators created.")


Pauli measurement operators created.


In [5]:
for basis, ops in pauli_measurements.items():
    total = ops[0] + ops[1]
    print(f"{basis} completeness check:")
    print(np.allclose(total, np.eye(2)))


Z completeness check:
True
X completeness check:
True
Y completeness check:
True


In [6]:
ket0 = np.array([[1],
                 [0]])
rho0 = ket0 @ ket0.T.conj()

for basis, ops in pauli_measurements.items():
    probs = [np.real(np.trace(P @ rho0)) for P in ops]
    print(f"{basis} measurement probabilities:", probs, "sum =", sum(probs))


Z measurement probabilities: [np.int64(1), np.int64(0)] sum = 1
X measurement probabilities: [np.float64(0.5), np.float64(0.5)] sum = 1.0
Y measurement probabilities: [np.float64(0.5), np.float64(0.5)] sum = 1.0


### Measurement Model Justification

Pauli projective measurements were selected for this tomography workflow due to their
simplicity, physical interpretability, and native support in most quantum simulators
and hardware platforms.

The completeness checks confirm that the projectors in each Pauli basis sum to the
identity operator, and numerical application of the Born rule verifies that all outcome
probabilities are non-negative and properly normalized.

While SIC POVMs offer informational completeness with fewer measurement outcomes, they
require non-standard measurement bases and more complex calibration. Given the goal of
building a reproducible and scalable baseline tomography pipeline, Pauli projective
measurements provide a clear and robust starting point.


In [7]:
import numpy as np

states = {
    "|0>": np.array([[1], [0]], dtype=complex),
    "|1>": np.array([[0], [1]], dtype=complex),
    "|+>": np.array([[1], [1]], dtype=complex) / np.sqrt(2),
    "|->": np.array([[1], [-1]], dtype=complex) / np.sqrt(2),
    "|+i>": np.array([[1], [1j]], dtype=complex) / np.sqrt(2),
}

print("Reference states created:", list(states.keys()))


Reference states created: ['|0>', '|1>', '|+>', '|->', '|+i>']


In [8]:
density_matrices = {}

for name, ket in states.items():
    rho = ket @ ket.conj().T
    density_matrices[name] = rho
    print(f"{name} density matrix:\n", rho, "\n")


|0> density matrix:
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

|1> density matrix:
 [[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]] 

|+> density matrix:
 [[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]] 

|-> density matrix:
 [[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]] 

|+i> density matrix:
 [[0.5+0.j  0. -0.5j]
 [0. +0.5j 0.5+0.j ]] 



In [9]:
import numpy as np
import plotly.graph_objects as go
from fractions import Fraction

_CUBE_FACES = (
    (0, 1, 2), (0, 2, 3),
    (4, 5, 6), (4, 6, 7),
    (0, 1, 5), (0, 5, 4),
    (1, 2, 6), (1, 6, 5),
    (2, 3, 7), (2, 7, 6),
    (3, 0, 4), (3, 4, 7)
)

def _phase_to_pi_string(angle_rad: float) -> str:
    if np.isclose(angle_rad, 0.0):
        return "0"
    multiple = angle_rad / np.pi
    frac = Fraction(multiple).limit_denominator(16)
    n, d = frac.numerator, frac.denominator
    sign = "-" if n < 0 else ""
    n = abs(n)
    if d == 1:
        mag = f"{n}" if n != 1 else ""
    else:
        mag = f"{n}/{d}"
    return f"{sign}{mag}π" if mag else f"{sign}π"

def plot_density_matrix_histogram(rho, basis_labels=None, title="Density matrix"):
    rho = np.asarray(rho)
    dim = rho.shape[0]
    mags = np.abs(rho)
    phases = np.angle(rho)

    if basis_labels is None:
        basis_labels = [str(i) for i in range(dim)]

    meshes = []
    colorbar_added = False

    for i in range(dim):
        for j in range(dim):
            h = mags[i, j]
            phase = phases[i, j]

            x0, x1 = i - 0.45, i + 0.45
            y0, y1 = j - 0.45, j + 0.45

            vertices = (
                (x0, y0, 0), (x1, y0, 0), (x1, y1, 0), (x0, y1, 0),
                (x0, y0, h), (x1, y0, h), (x1, y1, h), (x0, y1, h)
            )

            x, y, z = zip(*vertices)
            ii, jj, kk = zip(*_CUBE_FACES)

            mesh = go.Mesh3d(
                x=x, y=y, z=z,
                i=ii, j=jj, k=kk,
                intensity=[phase] * len(vertices),
                colorscale="HSV",
                cmin=-np.pi,
                cmax=np.pi,
                showscale=not colorbar_added,
                colorbar=dict(
                    title="phase",
                    tickvals=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
                    ticktext=["-π", "-π/2", "0", "π/2", "π"]
                ) if not colorbar_added else None
            )

            meshes.append(mesh)
            colorbar_added = True

    fig = go.Figure(data=meshes)
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(title="i", tickvals=list(range(dim)), ticktext=basis_labels),
            yaxis=dict(title="j", tickvals=list(range(dim)), ticktext=basis_labels),
            zaxis=dict(title="|ρᵢⱼ|")
        )
    )
    fig.show()



In [10]:
plot_density_matrix_histogram(
    density_matrices["|+>"],
    basis_labels=["0", "1"],
    title="Density matrix of |+> state"
)


## Reference State Preparation

The following single-qubit reference states were used for tomography.

| State | Circuit Description |
|------|---------------------|
| |0⟩ | Initialized in the computational basis |
| |1⟩ | X gate applied to |0⟩ |
| |+⟩ | Hadamard gate applied to |0⟩ |
| |−⟩ | X gate followed by Hadamard gate |
| |+i⟩ | Hadamard gate followed by S (phase) gate |


In [11]:
SHOTS = 2000
np.random.seed(42)


In [22]:
def sample_measurement_counts(rho, projectors, shots):
    probs = [np.real(np.trace(P @ rho)) for P in projectors]
    probs = np.maximum(probs, 0.0)
    probs = probs / np.sum(probs)
    counts = np.random.multinomial(shots, probs)
    return counts, probs


In [23]:
rho_plus = density_matrices["|+>"]

measurement_results = {}

for basis, ops in pauli_measurements.items():
    counts, probs = sample_measurement_counts(rho_plus, ops, SHOTS)
    measurement_results[basis] = {
        "counts": counts,
        "probs": probs
    }
    print(f"{basis} basis counts:", counts)
    print(f"{basis} basis probs :", probs)
    print()


Z basis counts: [1028  972]
Z basis probs : [0.5 0.5]

X basis counts: [2000    0]
X basis probs : [1. 0.]

Y basis counts: [1026  974]
Y basis probs : [0.5 0.5]



In [14]:
from pathlib import Path
import json

DATA_DIR = Path("data/single_qubit")
DATA_DIR.mkdir(parents=True, exist_ok=True)

for state_name, rho in density_matrices.items():
    state_dir = DATA_DIR / state_name.replace("|", "").replace(">", "")
    state_dir.mkdir(exist_ok=True)

    state_data = {}

    for basis, ops in pauli_measurements.items():
        counts, probs = sample_measurement_counts(rho, ops, SHOTS)
        state_data[basis] = {
            "counts": counts,
            "probs": probs
        }

    np.save(state_dir / "measurements.npy", state_data)

metadata = {
    "measurement_model": "Pauli projective",
    "bases": ["X", "Y", "Z"],
    "shots": SHOTS,
    "seed": 42
}

with open(DATA_DIR / "metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("Task 3 dataset saved")


Task 3 dataset saved


In [24]:
def expectation_from_counts(counts):
    return (counts[0] - counts[1]) / np.sum(counts)


In [25]:
expvals = {}

for basis in ["X", "Y", "Z"]:
    counts = measurement_results[basis]["counts"]   # <- this works now
    expvals[basis] = expectation_from_counts(counts)

print("Expectation values:")
for k, v in expvals.items():
    print(f"<{k}> = {v:.3f}")


Expectation values:
<X> = 1.000
<Y> = 0.026
<Z> = 0.028


In [26]:
I = np.eye(2)
X = np.array([[0, 1],
              [1, 0]])
Y = np.array([[0, -1j],
              [1j, 0]])
Z = np.array([[1, 0],
              [0, -1]])
def reconstruct_density_matrix(expvals):
    """
    Reconstruct a single-qubit density matrix from Pauli expectation values
    using linear inversion:

    rho = 1/2 ( I + <X>X + <Y>Y + <Z>Z )
    """
    rho = 0.5 * (
        I
        + expvals["X"] * X
        + expvals["Y"] * Y
        + expvals["Z"] * Z
    )
    return rho


In [27]:
rho_reconstructed = 0.5 * (
    I
    + expvals["X"] * X
    + expvals["Y"] * Y
    + expvals["Z"] * Z
)

print("Reconstructed density matrix:")
print(rho_reconstructed)


Reconstructed density matrix:
[[0.514+0.j    0.5  -0.013j]
 [0.5  +0.013j 0.486+0.j   ]]


In [28]:
plot_density_matrix_histogram(
    rho_reconstructed,
    basis_labels=["0", "1"],
    title="Reconstructed density matrix (|+⟩)"
)


In [29]:
plot_density_matrix_histogram(
    density_matrices["|+>"],
    basis_labels=["0", "1"],
    title="True density matrix (|+⟩)"
)


In [30]:
def fidelity_pure(rho_true, rho_est):
    return np.real(np.trace(rho_true @ rho_est))


In [31]:
from numpy.linalg import svd

def trace_distance(rho_true, rho_est):
    svals = svd(rho_true - rho_est, compute_uv=False)
    return 0.5 * np.sum(svals)


In [32]:
rho_true = density_matrices["|+>"]

F = fidelity_pure(rho_true, rho_reconstructed)
D = trace_distance(rho_true, rho_reconstructed)

print(f"Fidelity        : {F:.4f}")
print(f"Trace distance  : {D:.4f}")


Fidelity        : 1.0000
Trace distance  : 0.0191


## Validation Results

For the |+⟩ state reconstructed using Pauli projective measurements:

- Fidelity was close to unity, indicating accurate reconstruction.
- Trace distance remained small, with deviations dominated by shot noise.
- Increasing the number of shots improves reconstruction quality.

Shot noise and finite sampling are the primary error sources in this experiment.


In [33]:
import os
os.makedirs("data/single_qubit", exist_ok=True)

results = {}

for name, rho_true in density_matrices.items():
    measurement_results = {}

    for basis, ops in pauli_measurements.items():
        counts, _ = sample_measurement_counts(rho_true, ops, SHOTS)
        measurement_results[basis] = counts

    expectations = {
        b: expectation_from_counts(c)
        for b, c in measurement_results.items()
    }

    rho_est = reconstruct_density_matrix(expectations)

    F = fidelity_pure(rho_true, rho_est)
    D = trace_distance(rho_true, rho_est)

    results[name] = {
        "expectations": expectations,
        "rho_reconstructed": rho_est,
        "fidelity": F,
        "trace_distance": D
    }

    np.save(f"data/single_qubit/{name}_rho_true.npy", rho_true)
    np.save(f"data/single_qubit/{name}_rho_reconstructed.npy", rho_est)

    print(f"{name}: Fidelity = {F:.4f}, Trace distance = {D:.4f}")

np.save("data/single_qubit/summary_results.npy", results)


|0>: Fidelity = 1.0000, Trace distance = 0.0187
|1>: Fidelity = 1.0000, Trace distance = 0.0106
|+>: Fidelity = 1.0000, Trace distance = 0.0149
|->: Fidelity = 1.0000, Trace distance = 0.0245
|+i>: Fidelity = 1.0000, Trace distance = 0.0198


In [ ]:
state = "|+>"
plot_density_matrix_histogram(
    results[state]["rho_reconstructed"],
    basis_labels=["0", "1"],
    title=f"Reconstructed density matrix for {state}"
)


In [34]:
import pandas as pd


summary_data = {
    "State": [],
    "Fidelity": [],
    "Trace Distance": []
}

for state, info in results.items():
    summary_data["State"].append(state)
    summary_data["Fidelity"].append(info["fidelity"])
    summary_data["Trace Distance"].append(info["trace_distance"])

summary_df = pd.DataFrame(summary_data)
print("Validation Summary:")
display(summary_df)


for state, info in results.items():
    print(f"\n=== {state} ===")

    plot_density_matrix_histogram(
        density_matrices[state],
        basis_labels=["0", "1"],
        title=f"True density matrix ({state})"
    )

    plot_density_matrix_histogram(
        info["rho_reconstructed"],
        basis_labels=["0", "1"],
        title=f"Reconstructed density matrix ({state})"
    )


Validation Summary:


,State,Fidelity,Trace Distance
0,|0>,1.0,0.018682
1,|1>,1.0,0.010607
2,|+>,1.0,0.014866
3,|->,1.0,0.024546
4,|+i>,1.0,0.019780



=== |0> ===



=== |1> ===



=== |+> ===



=== |-> ===



=== |+i> ===


## Task 5 · Validation and Reporting

### 1. Validation Metrics
We reconstructed the density matrices for the single-qubit reference states (|0⟩, |1⟩, |+⟩, |−⟩, |+i⟩) and compared them to the true density matrices using fidelity and trace distance.  

| State | Fidelity | Trace Distance |
|-------|----------|----------------|
| |0⟩   | 1.0000   | 0.0187         |
| |1⟩   | 1.0000   | 0.0106         |
| |+⟩   | 1.0000   | 0.0149         |
| |−⟩   | 1.0000   | 0.0245         |
| |+i⟩  | 1.0000   | 0.0198         |

The density-matrix histograms provide a visual confirmation of accurate reconstruction.

---

### 2. Sources of Error
- **Shot noise**: Finite number of measurements introduces statistical fluctuations in probabilities.  
- **Model mismatch**: Using Pauli projective measurements instead of SIC POVMs can introduce bias in certain bases.  
- **Simulator approximations**: Small numerical errors due to machine precision and matrix operations.

---

### 3. Mitigation Strategies
- Increasing the number of shots (`SHOTS`) to reduce statistical uncertainty.  
- Ensuring proper normalization of density matrices after reconstruction.  
- Cross-validating results with alternative reconstruction methods (e.g., linear inversion vs maximum likelihood).  

---

### 4. Reflection and Future Work
- **Tooling friction**: Installation of dependencies and configuration of simulation environments required careful attention, but using Python notebooks and Plotly visualization simplified validation.  
- **Open questions**: How SIC POVMs compare to Pauli projectors in noisy or multi-qubit setups.  

